In [75]:
# global imports

import pandas as pd
import os
# from google.colab import drive
import json
import re
import nltk
import string
from nltk.corpus import stopwords
from collections import Counter, defaultdict

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# load the dataset to the colab

drive.mount('/content/drive')
dataset_path = '/content/drive/MyDrive/yelp_dataset/'

reviews = dataset_path + "yelp_academic_dataset_review.json"
business = dataset_path + "yelp_academic_dataset_business.json"


dataset = dataset_path + "yelp_data.json"
data_cleaned = dataset_path + 'yelp_cleaned.json'

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

contraction_mapping = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "gonna": "going to",
    "gotta": "got to",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'll": "he will",
    "he's": "he is",
    "how'd": "how did",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'll": "I will",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it would",
    "it'll": "it will",
    "it's": "it is",
    "let's": "let us",
    "might've": "might have",
    "must've": "must have",
    "mustn't": "must not",
    "shan't": "shall not",
    "she'd": "she would",
    "she'll": "she will",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "that'll": "that will",
    "that's": "that is",
    "there's": "there is",
    "they'd": "they would",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'll": "we will",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'd": "what did",
    "what'll": "what will",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "where'd": "where did",
    "where'll": "where will",
    "where's": "where is",
    "who'd": "who did",
    "who'll": "who will",
    "who's": "who is",
    "who've": "who have",
    "why'd": "why did",
    "why'll": "why will",
    "why's": "why is",
    "won't": "will not",
    "would've": "would have",
    "wouldn't": "would not",
    "you'd": "you would",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have"
}

NameError: name 'drive' is not defined

In [ ]:
# **Note:** Only for one-time execution

# Condense the dataset to only Restuarents and Food stalls


food_related_items = ['Chicken Wings', 'Bakeries', 'Creperies', 'Asian Fusion', 'Dumplings', 'Chocolatiers & Shops', 'Indian', 'Hot Tub & Pool', 'Ethiopian', 'Hawaiian', 'Poke', 'Soup', 'Vietnamese', 'Waffles', 'Falafel', 'Vegetarian', 'Cheesesteaks', 'Caribbean', 'Pita', 'Sushi Bars', 'Soup', 'Halal', 'Turkish', 'Chinese', 'Japanese Curry', 'Breweries', 'Taiwanese', 'Russian', 'Thai', 'Afghan', 'Tex-Mex', 'Iberian', 'Peruvian', 'Salvadoran', 'Laotian', 'Korean', 'Mexican', 'Dim Sum', 'Hakka', 'Venezuelan', 'Pakistani', 'Malaysian', 'Brazilian', 'Colombian', 'Cajun/Creole', 'Bubble Tea', 'Kebab', 'Trinidadian', 'Cambodian', 'Japanese', 'Tapas Bars', 'Tapas/Small Plates', 'Greek', 'restaurants', 'food']

food_related_business_ids = set()
with open(business, "r") as f:
  for i in f:
    json_data = json.loads(i)
    if json_data["categories"] != None:
      for j in food_related_items:
        if j in json_data["categories"]:
          food_related_business_ids.add(json_data["business_id"])


def filter_specific_ids(file_name, out_file_name):
    out_file = open(out_file_name, "w+")
    start = True
    with open(file_name, "r") as f:
        out_file.write('[')
        for i in f:
            if json.loads(i)["business_id"] in food_related_business_ids:
              if start:
                  start = False
              else:
                  out_file.write(",")
              data = json.loads(i)
              del data["review_id"]
              del data["user_id"]
              del data["business_id"]
              del data["date"]
              out_file.write(json.dumps(data))
    out_file.write(']')
    out_file.close()


filter_specific_ids(reviews, dataset)

In [ ]:
# **Note:** Only for one-time execution

# cleanse the data
dataframe = pd.read_json(dataset)
stop_words = set(nltk.corpus.stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()

def cleanse_data(d):

  #convert to lower case
  o = d.lower()

  # replace contractions with full forms
  for j in contraction_mapping.keys():
    o = o.replace(j, contraction_mapping[j])

  # replace extra white spaces, special characters
  o = re.sub(r'\s+', ' ', o)
  o = re.sub(r'[^a-zA-Z0-9\s]', '', o)

  # remove all new lines
  o = o.replace("\n", "")

  # remove stop words and lemmatize the tokens
  tokens = o.split()
  cleaned_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
  o = ' '.join(cleaned_tokens)

  # remove punctuation
  o = o.translate(str.maketrans('', '', string.punctuation))
  return o

dataframe["text"] = dataframe["text"].apply(cleanse_data)
dataframe.to_json(data_cleaned)

In [ ]:
# Run to load the dataset

json_data = pd.read_json("./yelp_cleaned.json")
json_data

In [ ]:
## Topic modelling

In [ ]:
# Aspect Extraction
# https://www.enjoyalgorithms.com/blog/aspect-base-sentiment-analysis-in-python
# run the command below once
nltk.download('averaged_perceptron_tagger')

In [6]:
xpath_expression1 = " //sentence"
xpath_expression2 = "//sentence/aspectCategories/aspectCategory[1]"
df = pd.read_xml('Restaurants_Train_v2.xml', xpath = xpath_expression1)
df2 = pd.read_xml('Restaurants_Train_v2.xml', xpath = xpath_expression2)
# Extract category attribute from aspectCategories
# df['category'] = df['aspectCategories'].apply(lambda x: x['aspectCategory']['category'])

# Drop the aspectCategories column as it's no longer needed
# df.drop(columns='aspectCategories', inplace=True)

final_df = df.merge(df2, left_index=True, right_index=True)
final_df.dropna(axis = 1, inplace=True)
categories = final_df['category'].unique()

In [9]:
for category in categories:
    print(category, len(final_df[final_df['category'] == category]))
final_df

service 410
food 1157
anecdotes/miscellaneous 1081
ambience 260
price 133


,id,text,category,polarity
0,3121,But the staff was so horrible to us.,service,negative
1,2777,"To be completely fair, the only redeeming fact...",food,positive
2,1634,"The food is uniformly exceptional, with a very...",food,positive
3,2534,Where Gabriela personaly greets you and recomm...,service,positive
4,583,"For those that go once and don't enjoy it, all...",anecdotes/miscellaneous,positive
...,...,...,...,...
3036,1063,But that is highly forgivable.,anecdotes/miscellaneous,positive
3037,777,"From the appetizers we ate, the dim sum and ot...",food,positive
3038,875,"When we arrived at 6:00 PM, the restaurant was...",anecdotes/miscellaneous,neutral
3039,671,Each table has a pot of boiling water sunken i...,food,neutral


In [11]:
def create_ngrams(tokens: list, n: int) -> list:
    """Creates n-grams for the given token sequence.
  Args:
    tokens (list): a list of tokens as strings
    n (int): the length of n-grams to create

  Returns:
    list: list of tuples of strings, each tuple being one of the individual n-grams
  """
    # STUDENTS IMPLEMENT
    ngrams = []
    for i in range(len(tokens) + 1 - n):
        ngrams.append(tuple(tokens[i:i + n]))
    return ngrams

def is_special(token):
    return token == SENTENCE_BEGIN or token == SENTENCE_END

In [92]:
"""
filter df by food category:

ngram model dict will look like:
    (ngram tuple): {
        positive: val
        negative: val
        neutral: val
        conflict: val
    }
also count the number of reviews in each class to get the P(class)
for example, the probability that 'I like sushi' is positive is
P(positive)*(P(<s> | positive))*P(I | positive)*P(like | positive)*P(sushi | positive)*P(</s> | positive)
^ for a unigram model

probability of <s>  and </s> should be 1,


"""

class Ngram_NB:
    def __init__(self, n_gram):
        """Initializes an untrained LanguageModel
        Args:
          n_gram (int): the n-gram order of the language model to create
        """
        self.n = n_gram
        self.category_probabilities = None
        self.ngrams = None
        self.raw_vocab = None
        
    def train(self, df, text_col="text", category_col="category", label_col = "polarity", verbose: bool = False) -> None:
        """Trains the language model on the given data. Assumes that the given data
        has tokens that are white-space separated, has one sentence per line, and
        that the sentences begin with <s> and end with </s>
        Args:
          tokens (list): tokenized data to be trained on as a single list
          verbose (bool): default value False, to be used to turn on/off debugging prints
        """
        # initialize class variables
        self.ngrams = dict()
        self.category_probabilities = defaultdict(lambda: defaultdict(int))
        
        # get the list of all tokens to determine rare words
        tokens = tokenize(df, text_col, self.n)
        self.raw_counts = Counter(tokens)
        
        # get the list of categories
        categories = df[category_col].unique()
        
        # for each category, filter the rows
        for category in categories:
            # defaultdict used to easily add elements, initialize to 1 for smoothing purposes
            self.ngrams[category] = defaultdict(lambda: defaultdict(lambda: 1)) 
            category_df = df[df[category_col] == category]
            # for each row, tokenize the sentence and create ngrams
            # THIS IS SLOW AND CAN PROBABLY BE IMPROVED
            for index, row in category_df.iterrows():
                sent_tokens = tokenize_line(row[text_col], self.n)
                filtered_sent_tokens = self.smooth_tokens(sent_tokens)
                ngrams = create_ngrams(filtered_sent_tokens, self.n)
                polarity = row['polarity']
                
                # update class counts for this category
                self.category_probabilities[category][polarity] += 1
                
                # for each ngram in the set of created ngrams, update the ngram counts for this category
                for ngram in ngrams:
                    # defaultdict should create these entries if they don't exist
                    self.ngrams[category][ngram][polarity] += 1
        if verbose:
            print(self.category_probabilities)
   
    def smooth_tokens(self, tokens: list):
        """Smooths a list of tokens by replacing rare words with UNK
        """
        if not self.raw_counts:
            raise ValueError("Model not yet trained")
        return [token if is_special(token) or self.raw_counts[token] > 1 else UNK for token in tokens]
    
    def score(self, input_string: str, category: str = 'food', verbose = False) -> float:
        """Calculates the probability scores for each polarity for a given string representing a single sequence of tokens.
        Args:
          sentence_tokens (list): a tokenized sequence to be scored by this model

        Returns:
          str: the most likely class for this string
          list: list of tuples, with each tuple containing the polarity and the corresponding
        """
        # STUDENTS IMPLEMENT
        if not self.ngrams:
            raise ValueError("Model not yet trained")

        # default category is food
        # COULD CHANGE THIS TO DO SCORE ACROSS ALL CATEGORIES AND CHOOSE THE BEST PROBABILITY
        
        tokens = tokenize_line(input_string, self.n)

        # replace rare words with UNK
        smooth_tokens = self.smooth_tokens(tokens)
        
        ngrams = create_ngrams(smooth_tokens, self.n)
        polarities = self.category_probabilities[category].keys()
        category_total = sum(self.category_probabilities[category].values())
        scores = dict()
        
        for polarity in polarities:
            # probability string is <polarity> = 
            # P(positive) * PRODUCT(# occurrences of ngram in positive docs / total words in positive docs) for each ngram
            score = self.category_probabilities[category][polarity] / category_total # P(<polarity>)
            
            if verbose:
                print(f'probability of {polarity} = {score}')
            
            # get the total number of ngrams in this polarity
            # could calculate this value once during training
            polarity_count = 0
            for ngram, polarity_counts in self.ngrams[category].items():
                polarity_count += polarity_counts[polarity]
            
            if verbose:
                print(f'number of terms with polarity {polarity} = {polarity_count}')
            for ngram in ngrams:
                ngram_polarity_occurrence = self.ngrams[category][ngram][polarity]
                if verbose:
                    print(f'number of times {ngram} appeared in polarity = {ngram_polarity_occurrence} / {polarity_count}')
                score *= ngram_polarity_occurrence / polarity_count
            scores[polarity] = score
        label = max(scores, key=scores.get)
        return label, scores

In [96]:
model = Ngram_NB(2)
# tokens = tokenize(final_df, 'text', 3)
model.train(final_df, verbose=True)

model.score('From the terrible service, to the bland food, not to mention the unaccommodating managers, the overall experience was horrible.', verbose = True)

defaultdict(<function Ngram_NB.train.<locals>.<lambda> at 0x0000020A9328FF60>, {'service': defaultdict(<class 'int'>, {'negative': 164, 'positive': 200, 'conflict': 31, 'neutral': 15}), 'food': defaultdict(<class 'int'>, {'positive': 820, 'negative': 187, 'neutral': 86, 'conflict': 64}), 'anecdotes/miscellaneous': defaultdict(<class 'int'>, {'positive': 516, 'neutral': 350, 'negative': 188, 'conflict': 27}), 'ambience': defaultdict(<class 'int'>, {'positive': 143, 'negative': 62, 'conflict': 36, 'neutral': 19}), 'price': defaultdict(<class 'int'>, {'positive': 62, 'negative': 57, 'neutral': 6, 'conflict': 8})})
probability of positive = 0.7087294727744166
number of terms with polarity positive = 14172
number of times ('<s>', 'terrible') appeared in polarity = 1 / 14172
number of times ('terrible', 'service') appeared in polarity = 1 / 14172
number of times ('service', 'bland') appeared in polarity = 1 / 14172
number of times ('bland', 'food') appeared in polarity = 1 / 14172
number of 

('negative',
 {'positive': 2.1575309071993123e-44,
  'negative': 4.604793947059776e-40,
  'neutral': 1.6004882804404917e-43,
  'conflict': 3.183150669671307e-43})